In [ ]:
import os, sys
import tensorflow as tf
import pandas as pd 
import numpy as np 
import nltk
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
from autocorrect import Speller
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
nltk.download('stopwords')
nltk.download('wordnet')
# Download NLTK resources
nltk.download('punkt')


In [ ]:
print(tf.__version__)

In [ ]:
train_df = pd.read_csv(r'C:/Users/nikhi/Comment_toxicity/dataset/train.csv')
train_df.head()

In [ ]:
test_df = pd.read_csv(r'C:/Users/nikhi/Comment_toxicity/dataset/test.csv')
test_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
contraction_dict = {"arent't":"are not", "s":"is", "ain't":"are not","don't":"do not"}
contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))

In [ ]:
def expand_contractions(text, contraction_dict=contraction_dict):
    def replace(match):
        return contraction_dict[match.group(0)]
    return contraction_re.sub(replace, text)

df = train_df.copy()
df['comment_text'] = df['comment_text'].apply(lambda x: expand_contractions(x))


In [ ]:
df['comment_text']

In [ ]:
def clean_text(text):
    # Remove newline characters
    text = re.sub('\n', ' ', text)
    # Remove other unwanted characters or symbols
    text = re.sub('[^a-zA-Z0-9\s]', '', text)
    # Remove extra whitespaces
    text = re.sub('\s+', ' ', text).strip()
    
    return text.lower()
    
    

df['comment_text'] = df['comment_text'].apply(lambda x:clean_text(x))

In [ ]:
# def corrected_spelling(text):
#     spell = Speller(lang='en')
#     tokens = word_tokenize(text)
#     corrected_tokens = [spell(word) for word in tokens if word is not None]
#     corrected_text = " ".join(corrected_tokens)
#     return corrected_text

# df['comment_text'] = df['comment_text'].apply(lambda x: corrected_spelling(x))

In [ ]:
# def corrected_spelling(text):
#     if isinstance(text, str):
#         spell = SpellChecker()
#         tokens = nltk.word_tokenize(text)
#         corrected_tokens = [spell.correction(word) for word in tokens if word is not None]
#         corrected_text = " ".join(corrected_tokens)
#         return corrected_text
#     else:
#         return ''


# df['comment_text'] = df['comment_text'].apply(corrected_spelling)

In [ ]:
df['comment_text']

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):
    
        tokens = nltk.word_tokenize(text)
        # remove stopwords
        stop_words = set(stopwords.words('english'))
        filtered_text = [word for word in tokens if word.lower() not in stop_words]

        lemmatizer = WordNetLemmatizer()
        lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_text]

        preprocessed_text = " ".join(lemmatized_tokens)
        return preprocessed_text 
    else:
        return '' 


In [ ]:
df['comment_text'] = df['comment_text'].apply(lambda x:preprocess_text(x))

In [ ]:
df.head()

In [ ]:
df.drop('id', axis=1, inplace=True)

In [ ]:
df[df.columns[1:]].values

In [ ]:
X = df['comment_text']
y = df[df.columns[1:]].values

In [ ]:
MAX_FEATURES = 200000
vecotorizer = TextVectorization(
    max_tokens=MAX_FEATURES,
    output_sequence_length=1800,
    output_mode='int'
)